# ανν

In [1]:
import os 
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score,make_scorer
from sklearn.metrics import mean_absolute_error 



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Dropout
from tensorflow.keras import optimizers


import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling


import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from mealpy.swarm_based import GWO 


from keras.callbacks import ModelCheckpoint

In [2]:
tf.random.set_seed(13)
tf.debugging.set_log_device_placement(False)

# Dataset Loading and Preprocessing

In [3]:
all_ds = pd.read_csv("dataset.csv")

In [4]:
all_ds

,Temperature,Catalyst,Time,Met-oil,Yield
0,40,3,105,15.0,59.8
1,60,3,105,10.5,92.6
2,80,1,105,10.5,88.2
3,80,5,105,10.5,60.6
4,60,5,150,10.5,85.2
5,40,5,105,10.5,59.4
6,60,3,105,10.5,92.6
7,40,3,150,10.5,58.8
8,60,3,150,15.0,75.6
9,60,1,105,6.0,69.6


In [5]:
all_ds.shape[0]

29

In [6]:
all_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Temperature  29 non-null     int64  
 1   Catalyst     29 non-null     int64  
 2   Time         29 non-null     int64  
 3   Met-oil      29 non-null     float64
 4   Yield        29 non-null     float64
dtypes: float64(2), int64(3)
memory usage: 1.3 KB


In [7]:
all_ds.isna().sum()

Temperature    0
Catalyst       0
Time           0
Met-oil        0
Yield          0
dtype: int64

## Split Data

In [8]:
target = "Yield"
input_data=all_ds.drop(columns = [target])
target_data=all_ds[target]

In [9]:
input_data = input_data.values
target_data = target_data.values.reshape(-1,1)

In [10]:
train_input, test_input, train_target, test_target= train_test_split(input_data, target_data, test_size=0.2, random_state=66)

print(f"Train Input: {train_input.shape[0]}")
print(f"Train Target: {train_target.shape[0]}")
print(f"Test Input: {test_input.shape[0]}")
print(f"Test Target: {test_target.shape[0]}")

Train Input: 23
Train Target: 23
Test Input: 6
Test Target: 6


## Scale Data

In [11]:
sc_x=StandardScaler()
train_input=sc_x.fit_transform(train_input)
test_input=sc_x.transform(test_input)
input_data = sc_x.transform(input_data)

sc_y=StandardScaler()
train_target=sc_y.fit_transform(train_target)
test_target=sc_y.transform(test_target)
target_data=sc_y.transform(target_data)

In [12]:
def get_preds(test_target, test_target_predict):
    test_target=pd.DataFrame(test_target)
    test_target.rename(columns={0:'Actual'}, inplace=True)
    test_target_predict=pd.DataFrame(test_target_predict)
    test_target_predict.rename(columns={0:'Predicted'}, inplace=True)
    predictions=pd.concat([test_target, test_target_predict], axis=1)
    return predictions

# Create Model

In [45]:
activation = 'relu'


def model_one_three_HL():
    
    model = Sequential()
    
    model.add(Dense(500, input_dim = train_input.shape[1], kernel_initializer='normal', activation=activation))
    
    model.add(Dense(400, kernel_initializer='normal', activation=activation))
    model.add(Dense(300, kernel_initializer='normal', activation=activation))
    
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    
    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
    
    return model

model_one = model_one_three_HL()
model_one.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 500)               2500      
                                                                 
 dense_21 (Dense)            (None, 400)               200400    
                                                                 
 dense_22 (Dense)            (None, 300)               120300    
                                                                 
 dense_23 (Dense)            (None, 1)                 301       
                                                                 
Total params: 323,501
Trainable params: 323,501
Non-trainable params: 0
_________________________________________________________________


# Create Checkpoints

In [46]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

# Train Model

In [47]:
import time
start_time=time.time()


model_one.fit(train_input, train_target, epochs=500, batch_size=32, validation_split = 0.1, callbacks=callbacks_list)


elapsed_time=time.time()-start_time
print(f'executed in {elapsed_time} seconds or {elapsed_time*1000} milliseconds')

Epoch 1/500
1/1 [==============================] - ETA: 0s - loss: 0.7818 - mean_absolute_error: 0.7818
Epoch 1: val_loss improved from inf to 1.49924, saving model to Weights-001--1.49924.hdf5
1/1 [==============================] - 1s 891ms/step - loss: 0.7818 - mean_absolute_error: 0.7818 - val_loss: 1.4992 - val_mean_absolute_error: 1.4992
Epoch 2/500
1/1 [==============================] - ETA: 0s - loss: 0.7619 - mean_absolute_error: 0.7619
Epoch 2: val_loss improved from 1.49924 to 1.48321, saving model to Weights-002--1.48321.hdf5
1/1 [==============================] - 0s 58ms/step - loss: 0.7619 - mean_absolute_error: 0.7619 - val_loss: 1.4832 - val_mean_absolute_error: 1.4832
Epoch 3/500
1/1 [==============================] - ETA: 0s - loss: 0.7467 - mean_absolute_error: 0.7467
Epoch 3: val_loss improved from 1.48321 to 1.46454, saving model to Weights-003--1.46454.hdf5
1/1 [==============================] - 0s 59ms/step - loss: 0.7467 - mean_absolute_error: 0.7467 - val_loss: 

# Test Set Accuracy

In [48]:
test_target_pred = model_one.predict(test_input)

1/1 [==============================] - 0s 80ms/step


In [49]:
r2_score(test_target, test_target_pred)

0.09853552446653302

In [50]:
test_target_unnorm = sc_y.inverse_transform(test_target)
test_target_pred_unnorm = sc_y.inverse_transform(test_target_pred.reshape(-1,1))
ann_prediction_test=get_preds(test_target_unnorm, test_target_pred_unnorm)
ann_prediction_test

,Actual,Predicted
0,75.6,78.295654
1,92.6,90.713890
2,69.6,86.868980
3,59.4,78.295654
4,89.6,84.630249
5,85.2,78.295654


In [51]:
ann_prediction_test["Actual"].corr(ann_prediction_test["Predicted"])

0.48013557568177967

In [52]:
from math import sqrt
print("TEST Mean absolute error :", mean_absolute_error(test_target_unnorm, test_target_pred_unnorm))
print("TEST Mean squared error :",mean_squared_error(test_target_unnorm, test_target_pred_unnorm))
print("TEST Root Mean squared error :",sqrt(mean_squared_error(test_target_unnorm, test_target_pred_unnorm)))

TEST Mean absolute error : 8.77008260091146
TEST Mean squared error : 123.07596878687002
TEST Root Mean squared error : 11.093960915149738


In [ ]:
# Fitness Function
# Upper and lower bounds
# Min or max
# Number of dimensions (Number od variables)

# Hyperparameters to optimize
# 1. Batch size : [2, 32]
# 2. Epoch : [200, 2000]
# 3. Optimizer : ['SGD', 'RMSprop', 'Adam', 'AdamW', 'Adadelta', 'Adagrad', 'Adamax', 'Adafactor', 'Nadam', 'Ftrl']
# 4. Learning rate : [0.01, 0.5]
# 5. Kernel Initializer : ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
# 6. Activation : ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
# 7. Number of hidden units : [5, 100]



# LABEL ENCODER
Opt_Encoder = LabelEncoder()
Opt_Encoder.fit(['SGD', 'RMSprop', 'Adam', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl'])

Act_Encoder = LabelEncoder()
Act_Encoder.fit(['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'])

Ken_Encoder = LabelEncoder()
Ken_Encoder.fit(['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'])


def decode_solution(solution):
    # ==============================CONSTRAINTS=============================
    batch_size = 2**int(solution[0])
    epoch = 10*int(solution[1])
    opt_integer = int(solution[2])
    optimizer = Opt_Encoder.inverse_transform([opt_integer])[0]
    learning_rate = solution[3]
    ken_init_integer = int(solution[4])
    ken_initializer = Ken_Encoder.inverse_transform([ken_init_integer])[0]
    act_integer = int(solution[5])
    activation = Act_Encoder.inverse_transform([act_integer])[0]
    number_hidden_units = int(solution[6])
    
    return [batch_size, epoch, optimizer, learning_rate, ken_initializer, activation, number_hidden_units]





def objective_function(solution):
    # ==============================CONSTRAINTS=============================
    batch_size, epoch, optimizer, learning_rate, ken_initializer, activation, number_hidden_units = decode_solution(solution)
    


    # ==============================CREATE MODEL=============================
    def model_one_three_HL():

        model = Sequential()

        model.add(Dense(number_hidden_units, input_dim = train_input.shape[1], kernel_initializer=ken_initializer, activation=activation))
        model.add(Dense(1, kernel_initializer=ken_initializer, activation=activation))

        opt = getattr(optimizers, optimizer)(learning_rate = learning_rate)
        model.compile(loss='mean_absolute_error', optimizer=opt, metrics=['mean_absolute_error'])

        return model

    model_one = model_one_three_HL()
    model_one.summary()
    # ==============================CHECKPOINTS=============================
    checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
    checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
    callbacks_list = [checkpoint]
    
    
    # ==============================TRAIN MODEL=============================
    model_one.fit(train_input, train_target, epochs=epoch, batch_size=batch_size, validation_split = 0.1, callbacks=callbacks_list)
    test_target_pred = model_one.predict(test_input)
    fitness = mean_squared_error(test_target, test_target_pred)
    
    return fitness
    
    
# batch_size
# epoch
# optimizer
# learning_rate
# ken_initializer
# activation
# number_hidden_units
    
#===================================BOUNDS
lb = [1,    2,  0,    0.01,  0,    0,     5   ]
ub = [3.99, 7, 7.99, 0.5,   7.99, 7.99,  200 ]




#===================================METAHEURISTIC
problem = {
    "obj_func": objective_function,
    "lb": lb,
    "ub": ub,
    "minmax": "min",
    "verbose": False,
}

epoch = 2
pop_size=50

model = GWO.BaseGWO(problem=problem, epoch=epoch, pop_size=pop_size)
model.solve()

#print(f"Best solution: {model.solution[0]}")

batch_size, epoch, optimizer, learning_rate, ken_initializer, activation, number_hidden_units = decode_solution (model.solution[0])

print (f"Batch Size: {batch_size},     Epoch: {epoch},    Optimizer: {optimizer},      Learning Rate: {learning_rate}") 
print(f"")
print (f"Kernel Initializer: {ken_initializer},     Activation: {activation},     Number of Hidden Units: {number_hidden_units}")




Model: "sequential_42"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_84 (Dense)            (None, 74)                370       
                                                                 
 dense_85 (Dense)            (None, 1)                 75        
                                                                 
Total params: 445
Trainable params: 445
Non-trainable params: 0
_________________________________________________________________
Epoch 1/40
 1/10 [==>...........................] - ETA: 4s - loss: 0.7153 - mean_absolute_error: 0.7153
Epoch 1: val_loss improved from inf to 1.39028, saving model to Weights-001--1.39028.hdf5
10/10 [==============================] - 1s 18ms/step - loss: 0.8310 - mean_absolute_error: 0.8310 - val_loss: 1.3903 - val_mean_absolute_error: 1.3903
Epoch 2/40
 1/10 [==>...........................] - ETA: 0s - loss: 0.4985 - mean_absolute_error: 0.4985

In [67]:
batch_size, epoch, optimizer, learning_rate, ken_initializer, activation, number_hidden_units = decode_solution (model.solution[0])

print (f"Batch Size: {batch_size},     Epoch: {epoch},    Optimizer: {optimizer},      Learning Rate: {learning_rate}") 
print(f"")
print (f"Kernel Initializer: {ken_initializer},     Activation: {activation},     Number of Hidden Units: {number_hidden_units}")

Batch Size: 4,     Epoch: 40,    Optimizer: Adagrad,      Learning Rate: 0.08228595570161022

Kernel Initializer: lecun_uniform,     Activation: linear,     Number of Hidden Units: 65


In [122]:
# ==============================CREATE MODEL=============================
def model_one_three_HL():

    model = Sequential()

    model.add(Dense(50, input_dim = train_input.shape[1], kernel_initializer=ken_initializer, activation=activation))
    model.add(Dense(number_hidden_units, kernel_initializer=ken_initializer, activation=activation))
    model.add(Dense(number_hidden_units, kernel_initializer=ken_initializer, activation=activation))
        
    model.add(Dense(1, kernel_initializer=ken_initializer, activation='linear'))

    opt = getattr(optimizers, optimizer)(learning_rate = learning_rate)
    model.compile(loss='mae', optimizer=opt, metrics=['mae'])

    return model

model_one = model_one_three_HL()
#model_one.summary()
# ==============================CHECKPOINTS=============================
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 0, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]


# ==============================TRAIN MODEL=============================
model_one.fit(train_input, train_target, epochs=50, batch_size=6, validation_split = 0.2, callbacks=callbacks_list, verbose=0)
test_target_pred = model_one.predict(test_input)
fitness = mean_squared_error(test_target, test_target_pred)









from math import sqrt

#===============================TEST SET ACCURACY
test_target_pred = model_one.predict(test_input)

test_target_unnorm = sc_y.inverse_transform(test_target)
test_target_pred_unnorm = sc_y.inverse_transform(test_target_pred.reshape(-1,1))
ann_prediction_test=get_preds(test_target_unnorm, test_target_pred_unnorm)

corr = ann_prediction_test["Actual"].corr(ann_prediction_test["Predicted"])

print("TEST Mean absolute error :", mean_absolute_error(test_target_unnorm, test_target_pred_unnorm))
print("TEST Mean squared error :",mean_squared_error(test_target_unnorm, test_target_pred_unnorm))
print("TEST Root Mean squared error :",sqrt(mean_squared_error(test_target_unnorm, test_target_pred_unnorm)))
print(f"R Squared: {r2_score(test_target, test_target_pred)}")
print(f"Correlation: {corr}")


1/1 [==============================] - 0s 21ms/step
TEST Mean absolute error : 9.202863820393882
TEST Mean squared error : 100.73446617070549
TEST Root Mean squared error : 10.03665612496042
R Squared: 0.2621746899082844
Correlation: 0.5172647097001106


In [124]:
train_target_pred = model_one.predict(train_input)
r2_score(train_target, train_target_pred)

1/1 [==============================] - 0s 21ms/step


0.35984078772435424

In [123]:
ann_prediction_test

,Actual,Predicted
0,75.6,80.929054
1,92.6,79.225380
2,69.6,82.421906
3,59.4,66.742363
4,89.6,86.177597
5,85.2,72.273163


# σβμ

# κρρ

# ξγβ

# ρφ

# ελμ